In [1]:
# Uncomment this line if you need to install the dependencies
#!pip install rasterio folium tqdm httpx rio-tiler geojson_pydantic cogeo-mosaic > /dev/null

In [2]:
import httpx

from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

from pystac_client import Client

from geojson_pydantic.features import Feature
from geojson_pydantic.geometries import Polygon

from pprint import pprint

In [3]:
stac_url = 'https://data.inpe.br/stac/'

client = Client.open(stac_url)

my_search = client.search(
    bbox=[-63,-15,-50,-10],
    datetime="2023-04-24/2023-04-24",
    collections=['S2_L2A_COG-1']
)

f"{my_search.matched()} items found"

'33 items found'

In [4]:
def getFeatures(item) -> Feature:
    
    return Feature(
        geometry=Polygon.from_bounds(*item.bbox),
        properties={
            "path": item.assets['tci'].href,
        }
    )


features = [ getFeatures(item).dict(exclude_none=True) for item in my_search.items() ]

features[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-57.000183, -10.039321319540866),
    (-56.260468, -10.039321319540866),
    (-56.260468, -9.045193112731958),
    (-57.000183, -9.045193112731958),
    (-57.000183, -10.039321319540866)]]},
 'properties': {'path': 'https://data.inpe.br/data/S2_L2A_COG/v001/21/L/WK/2023/4/S2B_TCI_20230424T140709_N0509_R110_T21LWK_20230424T164453.tif'}}

In [5]:
# from concurrent import futures
# from rio_tiler.io import COGReader

# # OR We could use Rasterio/rio-tiler

# def worker(src_path: str) -> Feature:
#     try:
#         with COGReader(src_path) as cog:
#             wgs_bounds = cog.geographic_bounds
#     except:
#         return {}

#     return Feature(
#         geometry=Polygon.from_bounds(*wgs_bounds),
#         properties={
#             "path": src_path,
#         }
#     )

# files = [ item.assets['tci'].href for item in my_search.items() ]

# with futures.ThreadPoolExecutor(max_workers=20) as executor:
#     features = [r.dict(exclude_none=True) for r in executor.map(worker, files) if r]

# features[0]

In [32]:
geojson = {'type': 'FeatureCollection', 'features': features}

bounds = featureBounds(geojson)

west, south, east, north = bounds

print(bounds)

m = Map(
    tiles="OpenStreetMap",
    location=((south + north) / 2, (west + east) / 2),
    zoom_start=1
)

m.fit_bounds(
    bounds=[
        [south, west],  # SouthWest
        [north, east],  # NorthEast
    ]
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)

geo_json.add_to(m)

# m


(-60.497375, -15.463528953850453, -56.260468, -9.040459945796002)


In [14]:
from rio_tiler.io import COGReader
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

print(features[0]['properties']['path'])

with COGReader(features[0]['properties']['path']) as cog:
    info = cog.info()
    print(info.minzoom)
    print(info.maxzoom)

https://data.inpe.br/data/S2_L2A_COG/v001/21/L/WK/2023/4/S2B_TCI_20230424T140709_N0509_R110_T21LWK_20230424T164453.tif
8
14


In [21]:
# We are creating the mosaicJSON using the features we created earlier
# by default MosaicJSON.from_feature will look in feature.properties.path to get the path of the dataset

mosaicdata = MosaicJSON.from_features(
    features, minzoom=info.minzoom, maxzoom=info.maxzoom)

mosaicjson_filename = "/home/wsq/code/python/titiler/titiler-mosaicJSON/mosaic_jsons/tci_mosaic.json"

with MosaicBackend(input=mosaicjson_filename, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    pprint(mosaic.mosaic_def.dict())


{'attribution': None,
 'bounds': (-60.497375, -15.463528953850453, -56.260468, -9.040459945796002),
 'center': (-58.378921500000004, -12.251994449823227, 8),
 'description': None,
 'maxzoom': 14,
 'minzoom': 8,
 'mosaicjson': '0.0.2',
 'name': None,
 'quadkey_zoom': 8,
 'tiles': {'21010321': ['https://data.inpe.br/data/S2_L2A_COG/v001/21/L/TK/2023/4/S2B_TCI_20230424T140709_N0509_R110_T21LTK_20230424T164453.tif'],
           '21010323': ['https://data.inpe.br/data/S2_L2A_COG/v001/21/L/TH/2023/4/S2B_TCI_20230424T140709_N0509_R110_T21LTH_20230424T164453.tif',
                        'https://data.inpe.br/data/S2_L2A_COG/v001/20/L/RP/2023/4/S2B_TCI_20230424T140709_N0509_R110_T20LRP_20230424T164453.tif',
                        'https://data.inpe.br/data/S2_L2A_COG/v001/20/L/RN/2023/4/S2B_TCI_20230424T140709_N0509_R110_T20LRN_20230424T164453.tif',
                        'https://data.inpe.br/data/S2_L2A_COG/v001/21/L/TJ/2023/4/S2B_TCI_20230424T140709_N0509_R110_T21LTJ_20230424T164453.tif',

In [28]:
# titiler_endpoint = "https://titiler.xyz"
titiler_endpoint = "http://0.0.0.0:8000"  # Local endpoint.

params = {
    "url": mosaicjson_filename
}

response = httpx.get(
    f"{titiler_endpoint}/mosaicjson/tilejson.json",
    params=params
).json()

print(response)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['http://0.0.0.0:8000/mosaicjson/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=%2Fhome%2Fwsq%2Fcode%2Fpython%2Ftitiler%2Ftitiler-mosaicJSON%2Fmosaic_jsons%2Ftci_mosaic.json'], 'minzoom': 8, 'maxzoom': 14, 'bounds': [-60.497375, -15.463528953850453, -56.260468, -9.040459945796002], 'center': [-58.378921500000004, -12.251994449823227, 8]}


In [33]:
tiles = TileLayer(
    tiles=response["tiles"][0],
    min_zoom=8,
    max_zoom=25,
    opacity=1,
    attr="Big INPE"
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)

m = Map(
    location=((south + north) / 2,(west + east) / 2),
    zoom_start=12
)

m.fit_bounds(
    bounds=[
        [south, west], # SouthWest
        [north, east], # NorthEast
    ]
)

tiles.add_to(m)
geo_json.add_to(m)
m